# Program Evaluasi Akurasi AI dalam Penilaian Kinerja Tim
## Penelitian: Perbandingan 5 Platform AI vs Penilaian HRD

**Konteks Penelitian:**
- 34 anggota tim proyek
- 611 lokasi kegiatan
- 5 Platform AI: ChatGPT, Claude, Gemini, Copilot, Specialized HR AI
- Gold Standard: Penilaian HRD riil

**Output:** CSV dengan penilaian kinerja dari 5 AI platforms + timestamp

## 1. Import Libraries

In [3]:
# Data manipulation
import pandas as pd
import numpy as np

# API clients
import openai  # ChatGPT
import anthropic  # Claude
import google.generativeai as genai  # Gemini
import requests  # For Copilot and HR AI APIs

# Time tracking
from datetime import datetime
import time

# System and environment
import os
from dotenv import load_dotenv

# JSON handling
import json

# Progress bar
from tqdm import tqdm

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Libraries imported successfully!")

ModuleNotFoundError: No module named 'openai'

## 2. Konfigurasi API Keys

**PENTING:** Simpan API keys Anda di file `.env` dengan format:
```
OPENAI_API_KEY=your_chatgpt_key_here
ANTHROPIC_API_KEY=your_claude_key_here
GOOGLE_API_KEY=your_gemini_key_here
COPILOT_API_KEY=your_copilot_key_here
HRAI_API_KEY=your_hr_ai_key_here
```

In [ ]:
# Load environment variables
load_dotenv()

# Configure API clients
openai.api_key = os.getenv('OPENAI_API_KEY')
anthropic_client = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

# API endpoints for Copilot and HR AI (sesuaikan dengan endpoint yang Anda gunakan)
COPILOT_API_URL = "https://api.copilot.example.com/v1/evaluate"  # Ganti dengan URL yang sesuai
HRAI_API_URL = "https://api.hrai.example.com/v1/assess"  # Ganti dengan URL yang sesuai

COPILOT_API_KEY = os.getenv('COPILOT_API_KEY')
HRAI_API_KEY = os.getenv('HRAI_API_KEY')

print("✅ API configurations loaded!")

## 3. Template Prompt Standar untuk Penilaian Kinerja

Prompt yang konsisten untuk semua platform AI agar hasil comparable.

In [ ]:
def create_evaluation_prompt(nama, usia, gender, minat_bidang, pengalaman, jumlah_lokasi):
    """
    Membuat prompt standar untuk evaluasi kinerja anggota tim.
    
    Parameters:
    -----------
    nama : str
        Nama anggota tim
    usia : int
        Usia anggota tim
    gender : str
        Gender (Laki-laki/Perempuan)
    minat_bidang : str
        Bidang keahlian/minat
    pengalaman : str
        Pengalaman kerja (misal: "3 tahun di bidang pelatihan")
    jumlah_lokasi : int
        Jumlah lokasi yang ditangani
    
    Returns:
    --------
    str : Prompt standar untuk evaluasi
    """
    
    prompt = f"""Anda adalah seorang HR evaluator yang profesional. Berdasarkan profil anggota tim berikut, berikan penilaian kinerja dalam skala 1-100.

PROFIL ANGGOTA TIM:
- Nama: {nama}
- Usia: {usia} tahun
- Gender: {gender}
- Bidang Keahlian: {minat_bidang}
- Pengalaman: {pengalaman}
- Jumlah Lokasi Ditangani: {jumlah_lokasi} lokasi

KONTEKS:
Ini adalah program pelatihan/coaching di bidang jasa dimana setiap lokasi melibatkan 15-35 peserta dalam 10 pertemuan.

TUGAS ANDA:
1. Berikan NILAI KINERJA dalam skala 1-100
2. Berikan ALASAN singkat (maksimal 3 kalimat) mengapa Anda memberikan nilai tersebut

FORMAT JAWABAN (WAJIB mengikuti format ini):
NILAI: [angka 1-100]
ALASAN: [penjelasan singkat]

Berikan penilaian yang objektif berdasarkan workload, pengalaman, dan keahlian."""
    
    return prompt

# Test prompt
test_prompt = create_evaluation_prompt(
    nama="Test User",
    usia=30,
    gender="Laki-laki",
    minat_bidang="Pelatihan Kepemimpinan",
    pengalaman="5 tahun sebagai trainer",
    jumlah_lokasi=25
)

print("📝 Contoh Prompt:")
print("="*80)
print(test_prompt)
print("="*80)

## 4. Fungsi untuk Memanggil Setiap Platform AI

In [ ]:
def extract_score_and_reason(response_text):
    """
    Ekstrak nilai dan alasan dari response AI.
    
    Parameters:
    -----------
    response_text : str
        Response dari AI
    
    Returns:
    --------
    tuple : (nilai, alasan)
    """
    try:
        # Cari pattern "NILAI: XX" atau "NILAI:XX"
        nilai = None
        alasan = ""
        
        lines = response_text.strip().split('\n')
        for line in lines:
            if 'NILAI:' in line.upper() or 'NILAI :' in line.upper():
                # Ekstrak angka
                parts = line.split(':')
                if len(parts) > 1:
                    nilai_str = parts[1].strip()
                    # Ambil angka pertama yang ditemukan
                    import re
                    match = re.search(r'\d+', nilai_str)
                    if match:
                        nilai = int(match.group())
            
            if 'ALASAN:' in line.upper() or 'ALASAN :' in line.upper():
                # Ekstrak alasan
                parts = line.split(':', 1)
                if len(parts) > 1:
                    alasan = parts[1].strip()
                # Jika alasan berlanjut di baris berikutnya
                idx = lines.index(line)
                if idx + 1 < len(lines) and not lines[idx + 1].startswith('NILAI'):
                    alasan += ' ' + ' '.join(lines[idx+1:])
                break
        
        # Jika tidak ditemukan dengan format, coba parsing lain
        if nilai is None:
            import re
            match = re.search(r'(\d+)\s*(?:/100)?', response_text)
            if match:
                nilai = int(match.group(1))
        
        if not alasan:
            alasan = response_text[:200]  # Ambil 200 karakter pertama sebagai alasan
        
        return nilai, alasan.strip()
    
    except Exception as e:
        print(f"⚠️ Error parsing response: {e}")
        return None, response_text[:200]

print("✅ Extraction function created!")

In [ ]:
def evaluate_with_chatgpt(prompt):
    """
    Evaluasi menggunakan ChatGPT (OpenAI GPT-4)
    """
    try:
        start_time = time.time()
        
        response = openai.ChatCompletion.create(
            model="gpt-4",  # atau "gpt-3.5-turbo" untuk lebih murah
            messages=[
                {"role": "system", "content": "You are an expert HR evaluator."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,  # Lower temperature untuk konsistensi
            max_tokens=500
        )
        
        end_time = time.time()
        response_time = end_time - start_time
        
        response_text = response.choices[0].message.content
        nilai, alasan = extract_score_and_reason(response_text)
        
        return {
            'platform': 'ChatGPT',
            'nilai': nilai,
            'alasan': alasan,
            'response_time': round(response_time, 2),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    except Exception as e:
        print(f"❌ ChatGPT Error: {e}")
        return {
            'platform': 'ChatGPT',
            'nilai': None,
            'alasan': f"Error: {str(e)}",
            'response_time': None,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

print("✅ ChatGPT function created!")

In [ ]:
def evaluate_with_claude(prompt):
    """
    Evaluasi menggunakan Claude (Anthropic)
    """
    try:
        start_time = time.time()
        
        message = anthropic_client.messages.create(
            model="claude-sonnet-4-5-20250929",  # atau model Claude lainnya
            max_tokens=500,
            temperature=0.3,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        end_time = time.time()
        response_time = end_time - start_time
        
        response_text = message.content[0].text
        nilai, alasan = extract_score_and_reason(response_text)
        
        return {
            'platform': 'Claude',
            'nilai': nilai,
            'alasan': alasan,
            'response_time': round(response_time, 2),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    except Exception as e:
        print(f"❌ Claude Error: {e}")
        return {
            'platform': 'Claude',
            'nilai': None,
            'alasan': f"Error: {str(e)}",
            'response_time': None,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

print("✅ Claude function created!")

In [ ]:
def evaluate_with_gemini(prompt):
    """
    Evaluasi menggunakan Google Gemini
    """
    try:
        start_time = time.time()
        
        model = genai.GenerativeModel('gemini-pro')
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.3,
                max_output_tokens=500,
            )
        )
        
        end_time = time.time()
        response_time = end_time - start_time
        
        response_text = response.text
        nilai, alasan = extract_score_and_reason(response_text)
        
        return {
            'platform': 'Gemini',
            'nilai': nilai,
            'alasan': alasan,
            'response_time': round(response_time, 2),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    except Exception as e:
        print(f"❌ Gemini Error: {e}")
        return {
            'platform': 'Gemini',
            'nilai': None,
            'alasan': f"Error: {str(e)}",
            'response_time': None,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

print("✅ Gemini function created!")

In [ ]:
def evaluate_with_copilot(prompt):
    """
    Evaluasi menggunakan Microsoft Copilot
    CATATAN: Sesuaikan dengan API endpoint Copilot yang Anda gunakan
    """
    try:
        start_time = time.time()
        
        headers = {
            'Authorization': f'Bearer {COPILOT_API_KEY}',
            'Content-Type': 'application/json'
        }
        
        payload = {
            'prompt': prompt,
            'temperature': 0.3,
            'max_tokens': 500
        }
        
        response = requests.post(COPILOT_API_URL, headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        
        end_time = time.time()
        response_time = end_time - start_time
        
        response_data = response.json()
        response_text = response_data.get('response', response_data.get('text', ''))
        nilai, alasan = extract_score_and_reason(response_text)
        
        return {
            'platform': 'Copilot',
            'nilai': nilai,
            'alasan': alasan,
            'response_time': round(response_time, 2),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    except Exception as e:
        print(f"❌ Copilot Error: {e}")
        return {
            'platform': 'Copilot',
            'nilai': None,
            'alasan': f"Error: {str(e)}",
            'response_time': None,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

print("✅ Copilot function created!")

In [ ]:
def evaluate_with_hr_ai(prompt):
    """
    Evaluasi menggunakan Specialized HR AI
    CATATAN: Sesuaikan dengan API endpoint HR AI yang Anda gunakan
    """
    try:
        start_time = time.time()
        
        headers = {
            'Authorization': f'Bearer {HRAI_API_KEY}',
            'Content-Type': 'application/json'
        }
        
        payload = {
            'prompt': prompt,
            'temperature': 0.3,
            'max_tokens': 500
        }
        
        response = requests.post(HRAI_API_URL, headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        
        end_time = time.time()
        response_time = end_time - start_time
        
        response_data = response.json()
        response_text = response_data.get('assessment', response_data.get('evaluation', ''))
        nilai, alasan = extract_score_and_reason(response_text)
        
        return {
            'platform': 'HR_AI',
            'nilai': nilai,
            'alasan': alasan,
            'response_time': round(response_time, 2),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    except Exception as e:
        print(f"❌ HR AI Error: {e}")
        return {
            'platform': 'HR_AI',
            'nilai': None,
            'alasan': f"Error: {str(e)}",
            'response_time': None,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

print("✅ HR AI function created!")

## 5. Fungsi Utama untuk Evaluasi dengan Semua Platform

In [ ]:
def evaluate_all_platforms(nama, usia, gender, minat_bidang, pengalaman, jumlah_lokasi):
    """
    Evaluasi menggunakan semua 5 platform AI secara berurutan.
    
    Returns:
    --------
    dict : Dictionary berisi hasil dari semua platform
    """
    
    print(f"\n{'='*80}")
    print(f"🔍 Mengevaluasi: {nama}")
    print(f"{'='*80}")
    
    # Buat prompt standar
    prompt = create_evaluation_prompt(nama, usia, gender, minat_bidang, pengalaman, jumlah_lokasi)
    
    # Dictionary untuk menyimpan hasil
    results = {
        'nama': nama,
        'usia': usia,
        'gender': gender,
        'minat_bidang': minat_bidang,
        'pengalaman': pengalaman,
        'jumlah_lokasi': jumlah_lokasi
    }
    
    # List platform functions
    platforms = [
        ('ChatGPT', evaluate_with_chatgpt),
        ('Claude', evaluate_with_claude),
        ('Gemini', evaluate_with_gemini),
        ('Copilot', evaluate_with_copilot),
        ('HR_AI', evaluate_with_hr_ai)
    ]
    
    # Evaluasi dengan setiap platform
    for platform_name, eval_func in platforms:
        print(f"\n⏳ Evaluating with {platform_name}...", end=' ')
        result = eval_func(prompt)
        
        # Simpan hasil ke dictionary dengan prefix platform
        results[f'{platform_name}_nilai'] = result['nilai']
        results[f'{platform_name}_alasan'] = result['alasan']
        results[f'{platform_name}_time'] = result['response_time']
        results[f'{platform_name}_timestamp'] = result['timestamp']
        
        if result['nilai'] is not None:
            print(f"✅ Done! (Nilai: {result['nilai']}, Time: {result['response_time']}s)")
        else:
            print(f"⚠️ Error!")
        
        # Delay kecil untuk menghindari rate limiting
        time.sleep(1)
    
    print(f"\n{'='*80}")
    print(f"✅ Evaluasi selesai untuk {nama}!")
    print(f"{'='*80}\n")
    
    return results

print("✅ Main evaluation function created!")

## 6. Input Data Anggota Tim

### Metode 1: Input Manual (Satu per Satu)

In [ ]:
# Inisialisasi list untuk menyimpan semua hasil
all_results = []

# Fungsi untuk input satu anggota tim
def input_single_member():
    """
    Input data untuk satu anggota tim secara interaktif.
    """
    print("\n" + "="*80)
    print("📝 INPUT DATA ANGGOTA TIM")
    print("="*80)
    
    nama = input("1️⃣ Nama: ")
    usia = int(input("2️⃣ Usia: "))
    gender = input("3️⃣ Gender (Laki-laki/Perempuan): ")
    minat_bidang = input("4️⃣ Minat Bidang/Keahlian: ")
    pengalaman = input("5️⃣ Pengalaman (misal: '3 tahun sebagai trainer'): ")
    jumlah_lokasi = int(input("6️⃣ Jumlah Lokasi yang Ditangani: "))
    
    print("\n" + "="*80)
    print("🔄 Memulai evaluasi dengan 5 AI platforms...")
    print("="*80)
    
    # Evaluasi dengan semua platform
    result = evaluate_all_platforms(nama, usia, gender, minat_bidang, pengalaman, jumlah_lokasi)
    
    # Simpan hasil
    all_results.append(result)
    
    print(f"\n✅ Data untuk {nama} telah disimpan! Total data: {len(all_results)}")
    
    return result

print("✅ Input function ready!")
print("\nUntuk menambahkan anggota tim, jalankan: input_single_member()")

In [ ]:
# JALANKAN CELL INI untuk input anggota tim pertama
# Uncomment baris di bawah untuk mulai input

# result = input_single_member()

### Metode 2: Batch Input (Multiple Members)

In [ ]:
# Contoh: Input batch untuk beberapa anggota sekaligus
# Format: List of dictionaries

batch_members = [
    {
        'nama': 'Ahmad Wijaya',
        'usia': 32,
        'gender': 'Laki-laki',
        'minat_bidang': 'Pelatihan Kepemimpinan',
        'pengalaman': '5 tahun sebagai corporate trainer',
        'jumlah_lokasi': 25
    },
    {
        'nama': 'Siti Nurhaliza',
        'usia': 28,
        'gender': 'Perempuan',
        'minat_bidang': 'Soft Skills Training',
        'pengalaman': '3 tahun di bidang HR development',
        'jumlah_lokasi': 18
    },
    # Tambahkan anggota lainnya di sini...
]

def process_batch_members(members_list):
    """
    Process multiple members sekaligus.
    """
    print(f"\n🚀 Processing {len(members_list)} anggota tim...\n")
    
    for i, member in enumerate(members_list, 1):
        print(f"\n{'#'*80}")
        print(f"# Anggota {i}/{len(members_list)}")
        print(f"{'#'*80}")
        
        result = evaluate_all_platforms(
            nama=member['nama'],
            usia=member['usia'],
            gender=member['gender'],
            minat_bidang=member['minat_bidang'],
            pengalaman=member['pengalaman'],
            jumlah_lokasi=member['jumlah_lokasi']
        )
        
        all_results.append(result)
        
        # Delay antar anggota
        if i < len(members_list):
            print("\n⏸️ Waiting 3 seconds before next member...")
            time.sleep(3)
    
    print(f"\n{'='*80}")
    print(f"🎉 SELESAI! Total {len(all_results)} anggota telah dievaluasi!")
    print(f"{'='*80}\n")

print("✅ Batch processing function ready!")
print("\nUntuk memproses batch, jalankan: process_batch_members(batch_members)")

In [ ]:
# JALANKAN CELL INI untuk memproses batch members
# Uncomment baris di bawah untuk mulai batch processing

# process_batch_members(batch_members)

## 7. Export ke CSV

Export hasil evaluasi ke format CSV untuk analisis lebih lanjut.

In [ ]:
def export_to_csv(filename='ai_evaluation_results.csv'):
    """
    Export semua hasil evaluasi ke CSV file.
    
    Parameters:
    -----------
    filename : str
        Nama file CSV output
    """
    if len(all_results) == 0:
        print("⚠️ Tidak ada data untuk di-export!")
        return
    
    # Konversi ke DataFrame
    df = pd.DataFrame(all_results)
    
    # Urutkan kolom
    base_cols = ['nama', 'gender', 'usia', 'minat_bidang', 'pengalaman', 'jumlah_lokasi']
    
    platforms = ['ChatGPT', 'Claude', 'Gemini', 'Copilot', 'HR_AI']
    platform_cols = []
    for platform in platforms:
        platform_cols.extend([
            f'{platform}_nilai',
            f'{platform}_alasan',
            f'{platform}_time',
            f'{platform}_timestamp'
        ])
    
    # Reorder columns
    all_cols = base_cols + platform_cols
    df = df[all_cols]
    
    # Export to CSV
    df.to_csv(filename, index=False, encoding='utf-8-sig')  # utf-8-sig untuk Excel compatibility
    
    print(f"\n{'='*80}")
    print(f"✅ Data berhasil di-export ke: {filename}")
    print(f"📊 Total rows: {len(df)}")
    print(f"📋 Total columns: {len(df.columns)}")
    print(f"{'='*80}\n")
    
    # Tampilkan preview
    print("Preview data:")
    print(df[['nama', 'gender', 'usia', 'ChatGPT_nilai', 'Claude_nilai', 'Gemini_nilai', 'Copilot_nilai', 'HR_AI_nilai']].head())
    
    return df

print("✅ Export function ready!")
print("\nUntuk export ke CSV, jalankan: export_to_csv()")

In [ ]:
# JALANKAN CELL INI untuk export hasil ke CSV
# Uncomment baris di bawah untuk export

# df_results = export_to_csv('ai_evaluation_results.csv')

## 8. Analisis Cepat (Preview)

Preview singkat hasil evaluasi.

In [ ]:
def quick_analysis():
    """
    Analisis cepat dari hasil evaluasi.
    """
    if len(all_results) == 0:
        print("⚠️ Belum ada data untuk dianalisis!")
        return
    
    df = pd.DataFrame(all_results)
    
    print(f"\n{'='*80}")
    print("📊 QUICK ANALYSIS")
    print(f"{'='*80}\n")
    
    print(f"Total Anggota Tim Dievaluasi: {len(df)}")
    print(f"\nDistribusi Gender:")
    print(df['gender'].value_counts())
    
    print(f"\n{'='*80}")
    print("Statistik Nilai per Platform:")
    print(f"{'='*80}\n")
    
    platforms = ['ChatGPT', 'Claude', 'Gemini', 'Copilot', 'HR_AI']
    
    stats_data = []
    for platform in platforms:
        nilai_col = f'{platform}_nilai'
        if nilai_col in df.columns:
            nilai_data = df[nilai_col].dropna()
            if len(nilai_data) > 0:
                stats_data.append({
                    'Platform': platform,
                    'Mean': round(nilai_data.mean(), 2),
                    'Median': round(nilai_data.median(), 2),
                    'Std Dev': round(nilai_data.std(), 2),
                    'Min': nilai_data.min(),
                    'Max': nilai_data.max()
                })
    
    stats_df = pd.DataFrame(stats_data)
    print(stats_df.to_string(index=False))
    
    print(f"\n{'='*80}")
    print("Response Time per Platform (seconds):")
    print(f"{'='*80}\n")
    
    time_data = []
    for platform in platforms:
        time_col = f'{platform}_time'
        if time_col in df.columns:
            time_vals = df[time_col].dropna()
            if len(time_vals) > 0:
                time_data.append({
                    'Platform': platform,
                    'Avg Time': round(time_vals.mean(), 2),
                    'Min Time': round(time_vals.min(), 2),
                    'Max Time': round(time_vals.max(), 2)
                })
    
    time_df = pd.DataFrame(time_data)
    print(time_df.to_string(index=False))
    
    print(f"\n{'='*80}\n")

print("✅ Quick analysis function ready!")
print("\nUntuk melihat analisis cepat, jalankan: quick_analysis()")

In [ ]:
# JALANKAN CELL INI untuk melihat analisis cepat
# Uncomment baris di bawah

# quick_analysis()

## 9. Utility Functions

Fungsi-fungsi tambahan yang berguna.

In [ ]:
def clear_all_results():
    """
    Clear semua hasil evaluasi (reset).
    """
    global all_results
    confirm = input("⚠️ Yakin ingin menghapus semua hasil? (yes/no): ")
    if confirm.lower() == 'yes':
        all_results = []
        print("✅ Semua hasil telah dihapus!")
    else:
        print("❌ Pembatalan dibatalkan.")

def show_current_results():
    """
    Tampilkan ringkasan hasil yang sudah ada.
    """
    if len(all_results) == 0:
        print("📭 Belum ada hasil evaluasi.")
        return
    
    print(f"\n📊 Total hasil: {len(all_results)}")
    print("\nDaftar anggota yang sudah dievaluasi:")
    for i, result in enumerate(all_results, 1):
        print(f"{i}. {result['nama']} ({result['gender']}, {result['usia']} tahun) - {result['jumlah_lokasi']} lokasi")
    print()

def save_checkpoint(checkpoint_name='checkpoint.json'):
    """
    Simpan progress ke file JSON (untuk backup).
    """
    with open(checkpoint_name, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    print(f"✅ Checkpoint disimpan ke: {checkpoint_name}")

def load_checkpoint(checkpoint_name='checkpoint.json'):
    """
    Load progress dari file JSON.
    """
    global all_results
    try:
        with open(checkpoint_name, 'r', encoding='utf-8') as f:
            all_results = json.load(f)
        print(f"✅ Checkpoint loaded: {len(all_results)} hasil")
    except FileNotFoundError:
        print(f"❌ File {checkpoint_name} tidak ditemukan!")

print("✅ Utility functions ready!")

## 10. QUICK START GUIDE

### Cara Penggunaan:

#### **Opsi 1: Input Manual (Satu per Satu)**
```python
# 1. Input data anggota tim
result = input_single_member()

# 2. Ulangi untuk anggota lainnya
result = input_single_member()

# 3. Export ke CSV
df = export_to_csv('hasil_evaluasi.csv')

# 4. Lihat analisis cepat
quick_analysis()
```

#### **Opsi 2: Batch Processing**
```python
# 1. Definisikan batch members di cell sebelumnya
# 2. Process batch
process_batch_members(batch_members)

# 3. Export ke CSV
df = export_to_csv('hasil_evaluasi.csv')

# 4. Lihat analisis cepat
quick_analysis()
```

#### **Fungsi Utilitas:**
```python
show_current_results()  # Lihat daftar yang sudah dievaluasi
save_checkpoint()       # Backup progress
load_checkpoint()       # Restore dari backup
clear_all_results()     # Reset semua data
```

### **CATATAN PENTING:**
1. ⚙️ Pastikan API keys sudah dikonfigurasi di file `.env`
2. 🔌 Sesuaikan endpoint API untuk Copilot dan HR AI
3. 💰 Perhatikan usage limits dan biaya API
4. 📊 Total waktu untuk 34 anggota ≈ 15-20 menit (tergantung kecepatan API)
5. 💾 Gunakan `save_checkpoint()` secara berkala untuk backup